In [ ]:
import os
import matplotlib.pyplot as plt
from glambie.data.data_catalogue import DataCatalogue
from glambie.plot.plot_helpers import get_colours, add_labels_axlines_and_title
from glambie.plot.plot_helpers import plot_non_cumulative_timeseries_on_axis, plot_cumulative_timeseries_on_axis
from glambie.data import submission_system_interface

In [ ]:
# read bucket catalogue - files used by algorithms
catalogue_bucket = DataCatalogue.from_glambie_submission_system()

In [ ]:
# read original unedited files (from ox)
catalogue_unedited_submissions = DataCatalogue.from_json_file('/data/ox1/working/glambie/local_copies_of_submitted_data/original_files_pre_edits/meta.json')
catalogue_unedited_submissions.load_all_data()

In [ ]:
# Let's print all datasets in Alaska
for d in catalogue_unedited_submissions.datasets:
    print(f"{d.region.name} \t {d.data_group.name} \t {d.user_group}")

In [ ]:
plots_path = '/data/ox1/working/glambie/glambie_submissions_summary_plots'

After the conversion to the annual grid is fixed, we can also add this plotting function here

In [ ]:
for original_dataset in catalogue_unedited_submissions.datasets:

    print(f"{original_dataset.region.name} \t {original_dataset.data_group.name} \t {original_dataset.user_group} \t {original_dataset.unit}")
    
    # Find corresponding dataset in bucket
    for bf in catalogue_bucket.datasets:
        if (original_dataset.region.name == bf.region.name) and (original_dataset.user_group == bf.user_group):
            bucket_dataset = bf

    # Generate timeseries for plotting
    print(original_dataset.unit)
    original_dataset_in_mwe = original_dataset.convert_timeseries_to_unit_mwe()  # get dataset into mwe
    print(original_dataset_in_mwe.data)
    bucket_dataset_in_mwe = bucket_dataset.convert_timeseries_to_unit_mwe()
    
    # First we need to convert it to our evenly spaced grid
    # original_dataset_date_grid = original_dataset_in_mwe.convert_timeseries_to_monthly_grid()
    # original_dataset_annual = original_dataset_date_grid.convert_timeseries_to_annual_trends()
    # bucket_dataset_date_grid = bucket_dataset_in_mwe.convert_timeseries_to_monthly_grid()
    # bucket_dataset_annual = bucket_dataset_date_grid.convert_timeseries_to_annual_trends()

    # Convert to a longterm_trend
    # original_dataset_trend = original_dataset_date_grid.convert_timeseries_to_longterm_trend()
    # bucket_dataset_trend = bucket_dataset_date_grid.convert_timeseries_to_longterm_trend()
    
    # Comparison plot
    plot_errors=False # set to True if you want to plot error bars

    _, axes = plt.subplots(2, 1, figsize=(11, 6))
    colours = get_colours(3)

    # plot non-cumulative timeseries
    plot_non_cumulative_timeseries_on_axis(
        result_dataframe=original_dataset_in_mwe.data.as_dataframe(), ax=axes[0], colour=colours[0], plot_errors=plot_errors)
    # plot_non_cumulative_timeseries_on_axis(
        # result_dataframe=bucket_dataset_annual.data.as_dataframe(), ax=axes[0], colour=colours[1], plot_errors=plot_errors)
    # plot_non_cumulative_timeseries_on_axis(
        # result_dataframe=bucket_dataset_trend.data.as_dataframe(), ax=axes[0], colour=colours[2], plot_errors=plot_errors)
    plot_non_cumulative_timeseries_on_axis(
        result_dataframe=bucket_dataset_in_mwe.data.as_dataframe(), ax=axes[0], colour=colours[1], plot_errors=plot_errors)
    
    
    # plot cumulative timeseries
    plot_cumulative_timeseries_on_axis(
        timeseries=original_dataset_in_mwe, ax=axes[1], colour=colours[0], plot_errors=plot_errors, linestyle="-",
        timeseries_for_vertical_adjustment=None, label="Original dataset (as submitted): " + original_dataset_in_mwe.user_group)
    # plot_cumulative_timeseries_on_axis(
        # timeseries=bucket_dataset_annual, ax=axes[1], colour=colours[1], plot_errors=plot_errors, linestyle="-",
        # timeseries_for_vertical_adjustment=bucket_dataset_in_mwe, # vertically adjust to original dataset
        # label="Annual rates: " + bucket_dataset_annual.user_group)
    # plot_cumulative_timeseries_on_axis(
        # timeseries=bucket_dataset_trend, ax=axes[1], colour=colours[2], plot_errors=plot_errors, linestyle="-",
        # timeseries_for_vertical_adjustment=bucket_dataset_in_mwe, # vertically adjust to original dataset
        # label="Trend: " + bucket_dataset_trend.user_group)
    plot_cumulative_timeseries_on_axis(
        timeseries=bucket_dataset_in_mwe, ax=axes[1], colour=colours[1], plot_errors=plot_errors, linestyle="-",
        timeseries_for_vertical_adjustment=None, label="GlAMBIE dataset (edited): " + bucket_dataset_in_mwe.user_group)

    add_labels_axlines_and_title(
        axes=axes, unit=bucket_dataset.unit, legend_fontsize=9,
        title="")
    
    figname = str(original_dataset.region.name) + '_' + str(original_dataset.data_group.name) + '_' + str(original_dataset.user_group) + '.png'
    
    plt.tight_layout()
    plt.savefig(os.path.join(plots_path, figname))
    plt.close()